<a href="https://colab.research.google.com/github/kwhc441/mask_police/blob/main/color_tracking_by_PC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#やり方その１
import numpy as np
import cv2

cap = cv2.VideoCapture(0) #カメラを定義

def red_range(img): #赤色の領域をマスクする関数
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #BGRをHSV色空間に変換

#赤色の領域を閾値でフィルタリング
#OpenCVのHSV色空間では赤は0~30付近と150~180付近に分かれるが、
#181~255は0からの循環なので180を中心に範囲を取れば赤の閾値を一回で指定できる。たぶん。
    hsv_min = np.array([170,170,60]) #色相(Hue)、彩度(Saturation)、明度(Value)
    hsv_max = np.array([190,255,255])
    mask = cv2.inRange(hsv, hsv_min, hsv_max) #hsvの各ドットについてhsv_minからhsv_maxの範囲内ならtrue

    return mask

while( cap.isOpened() ): #カメラが使える限りループ

    ret, frame = cap.read() #カメラの情報を取得。frameに640x480x3の配列データが入る。
    frame_np = red_range(np.array(frame)) #frameデータをnp配列に変換。

#領域のカタマリである「ブロブ」を識別し、データを格納する。すごくありがたい機能。
    nLabels, labelimages, data, center = cv2.connectedComponentsWithStats(frame_np)

    blob_count = nLabels - 1 #ブロブの数。画面領域全体を1つのブロブとしてカウントするので、-1する。

    if blob_count >= 1: #ブロブが1つ以上存在すれば、画面全体を示すブロブデータを削除。
        data = np.delete(data, 0, 0)
        center = np.delete(center, 0, 0)

#認識したブロブの中で最大領域を持つもののインデックスを取得
    tbi = np.argmax(data[:, 4]) #target_blob_index

#最大ブロブの中心に青の円マークを直径10pix、太さ3pixで描く
    cv2.circle(frame,(int(center[tbi][0]),int(center[tbi][1])),10,(255,0,0),3)

#画像を表示する
    cv2.imshow('RaspiCam_Live',frame)

#キーが押されたら終了する
    if cv2.waitKey(1) != -1:
        break

#終了処理。カメラを解放し、表示ウィンドウを破棄。
cap.release()
cv2.destroyAllWindows()


In [ ]:
#やり方その２
import numpy as np
import cv2

# 色の設定
lower_light_pink = np.array([170,170,60])
upper_light_pink = np.array([190,255,255])

_LOWER_COLOR = lower_light_pink
_UPPER_COLOR = upper_light_pink

def tracking():
    cap = cv2.VideoCapture(0)

    filter = ParticleFilter()
    filter.initialize()

    while True:
        ret, frame = cap.read()

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Threshold the HSV image to get only a color
        mask = cv2.inRange(hsv, _LOWER_COLOR, _UPPER_COLOR)

        # Start Tracking
        y, x = filter.filtering(mask)
        frame = cv2.circle(frame, (int(x), int(y)), 10, (255, 0, 0), -1)

        # origin is upper left
        frame_size = frame.shape
        print ("position_x_rate")
        print (x/frame_size[1])
        print ("position_y_rate")
        print (y/frame_size[0])

        for i in range(filter.SAMPLEMAX):
            frame = cv2.circle(frame, (int(filter.X[i]), int(filter.Y[i])), 2, (0, 0, 255), -1)
        cv2.imshow("frame", frame)

        if cv2.waitKey(20) & 0xFF == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

class ParticleFilter:
    def __init__(self):
        self.SAMPLEMAX = 1000
        # frame.shape
        self.height, self.width = 480, 640

    def initialize(self):
        self.Y = np.random.random(self.SAMPLEMAX) * self.height
        self.X = np.random.random(self.SAMPLEMAX) * self.width

    # Need adjustment for tracking object velocity
    def modeling(self):
        self.Y += np.random.random(self.SAMPLEMAX) * 200 - 100 # 2:1
        self.X += np.random.random(self.SAMPLEMAX) * 200 - 100

    def normalize(self, weight):
        return weight / np.sum(weight)

    def resampling(self, weight):
        index = np.arange(self.SAMPLEMAX)
        sample = []

        # choice by weight 
        for i in range(self.SAMPLEMAX):
            idx = np.random.choice(index, p=weight)
            sample.append(idx)
        return sample

    def calcLikelihood(self, image):
        # white space tracking 
        mean, std = 250.0, 10.0
        intensity = []

        for i in range(self.SAMPLEMAX):
            y, x = self.Y[i], self.X[i]
            if y >= 0 and y < self.height and x >= 0 and x < self.width:
                intensity.append(image[int(y),int(x)])
            else:
                intensity.append(-1)

        # normal distribution
        weights = 1.0 / np.sqrt(2 * np.pi * std) * np.exp(-(np.array(intensity) - mean)**2 /(2 * std**2))
        weights[intensity == -1] = 0
        weights = self.normalize(weights)
        return weights

    def filtering(self, image):
        self.modeling()
        weights = self.calcLikelihood(image)
        index = self.resampling(weights)
        self.Y = self.Y[index]
        self.X = self.X[index]
        
        # return COG
        return np.sum(self.Y) / float(len(self.Y)), np.sum(self.X) / float(len(self.X))
tracking()
